# Melakukan Scrap Terhadap Gmail

In [2]:
import imaplib, email, os, re

user = 'kosar.candidate@gmail.com'
pasw = '123ewqASD'
imap_url = 'imap.gmail.com'
attachment_dir = 'C:/Users/PidiePie'

def get_body(msg): #fungsi extract body
    if msg.is_multipart():
        return get_body(msg.get_payload(0))
    else :
        return msg.get_payload(None,True)
    
def search(key,value,con): #fungsi search
    result, data = con.search(None,key,'"{}"'.format(value))
    return data

def get_attachment(msg): # fungsi untuk ngambil attachment
    for part in msg.walk():
        if part.get_content_maintype()=='multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
        fileName = part.get_filename()
        if bool(fileName):
            filePath = os.path.join(attachment_dir, fileName)
            with open(filePath, 'wb') as f:
                f.write(part.get_payload(decode=True))

con = imaplib.IMAP4_SSL(imap_url)
con.login(user,pasw)
con.select('INBOX')
result, data = con.search(None, '(FROM "noreply@ilmuonedata.com")') # ambil email

ids = data[0] # buat data jadi list.
id_list = ids.split() # pisahkan
latest_email_id = id_list[-1] # ambil yang trakhir

result, data = con.fetch(latest_email_id, "(RFC822)")  #email terakhir
raw = email.message_from_bytes(data[0][1])
get_attachment(raw) #ambil attachment
print('email berhasil di scrap!')
print('berikut clue yang diberikan:')
get_body(raw)# ambil body dimana sebagai clue
isibody = str(get_body(raw))
nilai=re.findall(r"\d+", isibody)
print('untuk hari',nilai[0],'clue yang dberikan',nilai[2])

email berhasil di scrap!
berikut clue yang diberikan:
untuk hari 5 clue yang dberikan 637


# Memproses Data dari Attachment email yang diberikan

In [3]:
import pandas as pd
import os
import pandas as pd

search_dir = r"C:\Users\PidiePie\email baru"
os.chdir(search_dir)
files = filter(os.path.isfile, os.listdir(search_dir))
files = [os.path.join(search_dir, f) for f in files] # add path to each file
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
dfs = pd.read_csv(files[0])
dfs.shape

total=0
clue= nilai[2] #insert clue
for x in dfs.iterrows():
    i = x[1][1]
    v = x[1][2]
    if clue in i:
        total += v
print('the total value from id contains '+str(clue)+' is '+str(total))

the total value from id contains 637 is 713


# Melakukan Load/Forward email atas hasil dari pengolahan data terhadap client

In [10]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os.path

day= str(nilai[1])
email = 'kosar.candidate@gmail.com'
pasw = '123ewqASD'
send_to_email = ['sally@ilmuonedata.com', 'fuad@ilmuonedata.com','optimize@ilmuonedata.com' ]
# send_to_email = ['kautsareko@gmail.com']
subject = '[email-scrapping] day '+ str(day) +' result'
message = 'the total value from id contains '+str(clue)+' is '+str(total) + '\n''''Today is the last day, and I have\
optimized the entire program code so that it can run fully effective. No longer need to fill variables manually. Once the code is executed, it will carry out the following process:
1. Scrap Gmail with the specifications of who is the sender, and the latest inbox.
2. taking the body text and downloading the attachment that contains the value of the given id
3. after that the attachment downloaded is calculated value with the given clue id
4. the results of processing will be made an email with the conditions that have been set by Ilmuone Data.
5. The last, the email will be forwarded to stakeholders that have been provided by IlmuOne Data.

You can see the latest program code in the attachment below or access the link https://github.com/kautsareko/Test-IlmuOneData.
Thank you for your attention.

Best Regards,

Alkautsar'''

file_location = 'C:\\Users\\PidiePie\\Final_Code_IlmuoneData_Test.ipynb'

msg = MIMEMultipart()
msg['From'] = email
msg['To'] = ",".join(send_to_email)
msg['Subject'] = subject

msg.attach(MIMEText(message, 'plain'))

filename= os.path.basename(file_location)
attachment = open (file_location, "rb")
part= MIMEBase('application','octet-stream')
part.set_payload((attachment).read())
encoders.encode_base64(part)
part.add_header('Content-Disposition',"attachment; filename= %s" % filename)

msg.attach(part)

server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(email, pasw)
text = msg.as_string()
server.sendmail(email, send_to_email, text)
server.quit
print('sukses di kirim!!!')

sukses di kirim!!!
